# Learning Data

In [1]:
from __future__ import print_function
import argparse
import os
import math
import numpy as np
import numpy.random as npr
import scipy.misc
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import matplotlib
from load_data import load_cifar10
import matplotlib.pyplot as plt

matplotlib.use('Agg')  # switch backend

HORSE_CATEGORY = 7

/anaconda3/envs/Python2.7/lib/python2.7/site-packages/ipykernel_launcher.py:17: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/anaconda3/envs/Python2.7/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/anaconda3/envs/Python2.7/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/anaconda3/envs/Python2.7/lib/python2.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/anaconda3/envs/Python2.7/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/anaconda3/envs/Python2.7/lib/python2.7/site-packages/ipykernel/

In [2]:
gpu = False
valid = False
# checkpoint = ""
checkpoint = "unet_k3_f32.pkl"
plot = False
colours = "colours/colour_kmeans24_cat7.npy"
model = "UNet"
kernel = 3
num_filters = 32
learn_rate = 0.001
# batch_size = 100
batch_size = 10
# epochs = 25
epochs = 5
seed = 0

In [3]:
######################################################################
# Data related code
######################################################################
def get_rgb_cat(xs, colours):
    """
    Get colour categories given RGB values. This function doesn't
    actually do the work, instead it splits the work into smaller
    chunks that can fit into memory, and calls helper function
    _get_rgb_cat

    Args:
      xs: float numpy array of RGB images in [B, C, H, W] format
      colours: numpy array of colour categories and their RGB values
    Returns:
      result: int numpy array of shape [B, 1, H, W]
    """
    if np.shape(xs)[0] < 100:
        return _get_rgb_cat(xs)
    batch_size = 100
    nexts = []
    for i in range(0, np.shape(xs)[0], batch_size):
        next = _get_rgb_cat(xs[i:i + batch_size, :, :, :], colours)
        nexts.append(next)
    result = np.concatenate(nexts, axis = 0)
    return result


def _get_rgb_cat(xs, colours):
    """
    Get colour categories given RGB values. This is done by choosing
    the colour in `colours` that is the closest (in RGB space) to
    each point in the image `xs`. This function is a little memory
    intensive, and so the size of `xs` should not be too large.

    Args:
      xs: float numpy array of RGB images in [B, C, H, W] format
      colours: numpy array of colour categories and their RGB values
    Returns:
      result: int numpy array of shape [B, 1, H, W]
    """
    num_colours = np.shape(colours)[0]
    xs = np.expand_dims(xs, 0)
    cs = np.reshape(colours, [num_colours, 1, 3, 1, 1])
    dists = np.linalg.norm(xs - cs, axis = 2)  # 2 = colour axis
    cat = np.argmin(dists, axis = 0)
    cat = np.expand_dims(cat, axis = 1)
    return cat


def get_cat_rgb(cats, colours):
    """
    Get RGB colours given the colour categories

    Args:
      cats: integer numpy array of colour categories
      colours: numpy array of colour categories and their RGB values
    Returns:
      numpy tensor of RGB colours
    """
    return colours[cats]


def process(xs, ys, max_pixel = 256.0):
    """
    Pre-process CIFAR10 images by taking only the horse category,
    shuffling, and have colour values be bound between 0 and 1

    Args:
      xs: the colour RGB pixel values
      ys: the category labels
      max_pixel: maximum pixel value in the original data
    Returns:
      xs: value normalized and shuffled colour images
      grey: greyscale images, also normalized so values are between 0 and 1
    """
    xs = xs / max_pixel
    xs = xs[np.where(ys == HORSE_CATEGORY)[0], :, :, :]
    npr.shuffle(xs)
    grey = np.mean(xs, axis = 1, keepdims = True)
    return (xs, grey)


def get_batch(x, y, batch_size):
    '''
    Generated that yields batches of data

    Args:
      x: input values
      y: output values
      batch_size: size of each batch
    Yields:
      batch_x: a batch of inputs of size at most batch_size
      batch_y: a batch of outputs of size at most batch_size
    '''
    N = np.shape(x)[0]
    assert N == np.shape(y)[0]
    for i in range(0, N, batch_size):
        batch_x = x[i:i + batch_size, :, :, :]
        batch_y = y[i:i + batch_size, :, :, :]
        yield (batch_x, batch_y)


def plot(input, gtlabel, output, colours, path):
    """
    Generate png plots of input, ground truth, and outputs

    Args:
      input: the greyscale input to the colourization CNN
      gtlabel: the grouth truth categories for each pixel
      output: the predicted categories for each pixel
      colours: numpy array of colour categories and their RGB values
      path: output path
    """
    grey = np.transpose(input[:10, :, :, :], [0, 2, 3, 1])
    gtcolor = get_cat_rgb(gtlabel[:10, 0, :, :], colours)
    predcolor = get_cat_rgb(output[:10, 0, :, :], colours)

    img = np.vstack([
        np.hstack(np.tile(grey, [1, 1, 1, 3])),
        np.hstack(gtcolor),
        np.hstack(predcolor)])
    scipy.misc.toimage(img, cmin = 0, cmax = 1).save(path)


In [4]:
######################################################################
# Torch Helper
######################################################################

def get_torch_vars(xs, ys, gpu = False):
    """
    Helper function to convert numpy arrays to pytorch tensors.
    If GPU is used, move the tensors to GPU.

    Args:
      xs (float numpy tenosor): greyscale input
      ys (int numpy tenosor): categorical labels
      gpu (bool): whether to move pytorch tensor to GPU
    Returns:
      Variable(xs), Variable(ys)
    """
    xs = torch.from_numpy(xs).float()
    ys = torch.from_numpy(ys).long()
    if gpu:
        xs = xs.cuda()
        ys = ys.cuda()
    return Variable(xs), Variable(ys)


def compute_loss(criterion, outputs, labels, batch_size, num_colours):
    """
    Helper function to compute the loss. Since this is a pixelwise
    prediction task we need to reshape the output and ground truth
    tensors into a 2D tensor before passing it in to the loss criteron.

    Args:
      criterion: pytorch loss criterion
      outputs (pytorch tensor): predicted labels from the model
      labels (pytorch tensor): ground truth labels
      batch_size (int): batch size used for training
      num_colours (int): number of colour categories
    Returns:
      pytorch tensor for loss
    """

    loss_out = outputs.transpose(1, 3) \
        .contiguous() \
        .view([batch_size * 32 * 32, num_colours])
    loss_lab = labels.transpose(1, 3) \
        .contiguous() \
        .view([batch_size * 32 * 32])
    return criterion(loss_out, loss_lab)


def run_validation_step(cnn, criterion, test_grey, test_rgb_cat, batch_size,
                        colour, plotpath = None):
    correct = 0.0
    total = 0.0
    losses = []
    for i, (xs, ys) in enumerate(get_batch(test_grey,
                                           test_rgb_cat,
                                           batch_size)):
        images, labels = get_torch_vars(xs, ys, gpu)
        outputs = cnn(images)

        val_loss = compute_loss(criterion,
                                outputs,
                                labels,
                                batch_size = batch_size,
                                num_colours = num_colours)
        losses.append(val_loss.data[0])

        _, predicted = torch.max(outputs.data, 1, keepdim = True)
        total += labels.size(0) * 32 * 32
        correct += (predicted == labels.data).sum()

    if plotpath:  # only plot if a path is provided
        plot(xs, ys, predicted.cpu().numpy(), colours, plotpath)

    val_loss = np.mean(losses)
    val_acc = 100 * correct / total
    return val_loss, val_acc

In [5]:
class MyConv2d(nn.Module):
    """
    Our simplified implemented of nn.Conv2d module for 2D convolution
    """

    def __init__(self, in_channels, out_channels, kernel_size, padding = None):
        super(MyConv2d, self).__init__()

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        if padding is None:
            self.padding = kernel_size // 2
        else:
            self.padding = padding
        self.weight = nn.parameter.Parameter(torch.Tensor(
            out_channels, in_channels, kernel_size, kernel_size))
        self.bias = nn.parameter.Parameter(torch.Tensor(out_channels))
        self.reset_parameters()

    def reset_parameters(self):
        n = self.in_channels * self.kernel_size * self.kernel_size
        stdv = 1. / math.sqrt(n)
        self.weight.data.uniform_(-stdv, stdv)
        self.bias.data.uniform_(-stdv, stdv)

    def forward(self, input):
        return F.conv2d(input, self.weight, self.bias, padding = self.padding)

    
class CNN(nn.Module):
    def __init__(self, kernel, num_filters, num_colours):
        super(CNN, self).__init__()
        padding = kernel // 2

        ############### YOUR CODE GOES HERE ###############
        self.downconv1 = nn.Sequential(
            MyConv2d(1, num_filters, kernel_size = kernel, padding = padding),
            nn.MaxPool2d(2),
            nn.BatchNorm2d(num_filters),
            nn.ReLU())

        self.downconv2 = nn.Sequential(
            MyConv2d(num_filters, num_filters * 2, kernel_size = kernel,
                      padding = padding),
            nn.MaxPool2d(2),
            nn.BatchNorm2d(num_filters * 2),
            nn.ReLU())

        self.rfconv = nn.Sequential(
            MyConv2d(num_filters * 2, num_filters * 2, kernel_size = kernel,
                      padding = padding),
            nn.BatchNorm2d(num_filters * 2),
            nn.ReLU())

        self.upconv1 = nn.Sequential(
            MyConv2d(num_filters * 2, num_filters, kernel_size = kernel,
                      padding = padding),
            nn.Upsample(scale_factor = 2),
            nn.BatchNorm2d(num_filters),
            nn.ReLU())
        self.upconv2 = nn.Sequential(
            MyConv2d(num_filters, num_colours, kernel_size = kernel, padding = padding),
            nn.Upsample(scale_factor = 2),
            nn.BatchNorm2d(num_colours),
            nn.ReLU())

        self.finalconv = MyConv2d(num_colours, num_colours, kernel_size = kernel)
        ###################################################

    def forward(self, x):
        self.out1 = self.downconv1(x)
        self.out2 = self.downconv2(self.out1)
        self.out3 = self.rfconv(self.out2)
        self.out4 = self.upconv1(self.out3)
        self.out5 = self.upconv2(self.out4)
        self.out_final = self.finalconv(self.out5)
        return self.out_final


In [6]:
class UNet(nn.Module):
    def __init__(self, kernel, num_filters, num_colours):
        super(UNet, self).__init__()
        padding = kernel // 2

        ############### YOUR CODE GOES HERE ###############
        self.downconv1 = nn.Sequential(
            MyConv2d(1, num_filters, kernel_size = kernel, padding = padding),
            nn.MaxPool2d(2),
            nn.BatchNorm2d(num_filters),
            nn.ReLU())

        self.downconv2 = nn.Sequential(
            MyConv2d(num_filters, num_filters * 2, kernel_size = kernel,
                      padding = padding),
            nn.MaxPool2d(2),
            nn.BatchNorm2d(num_filters * 2),
            nn.ReLU())

        self.rfconv = nn.Sequential(
            MyConv2d(num_filters * 2, num_filters * 2, kernel_size = kernel,
                      padding = padding),
            nn.BatchNorm2d(num_filters * 2),
            nn.ReLU())

        self.upconv1 = nn.Sequential(
            MyConv2d(num_filters * 2, num_filters, kernel_size = kernel,
                      padding = padding),
            nn.Upsample(scale_factor = 2),
            nn.BatchNorm2d(num_filters),
            nn.ReLU())
        self.upconv2 = nn.Sequential(
            MyConv2d(num_filters, num_colours, kernel_size = kernel, padding = padding),
            nn.Upsample(scale_factor = 2),
            nn.BatchNorm2d(num_colours),
            nn.ReLU())

        self.finalconv = MyConv2d(num_colours, num_colours, kernel_size = kernel)
        ###################################################

    def forward(self, x):
        ############### YOUR CODE GOES HERE ###############
        print("x: {}".format(x.shape))

        self.out1 = self.downconv1(x)
        print("out 1: {}".format(self.out1.shape))

        self.out2 = self.downconv2(self.out1)
        print("out 2: {}".format(self.out2.shape))

        self.out3 = self.rfconv(self.out2)
        print("out 3: {}".format(self.out3.shape))

        self.out4 = self.upconv1(self.out3)
        print("out 4: {}".format(self.out4.shape))
        # self.out4 = torch.cat((self.out4, self.out2), 1)

        self.out5 = self.upconv2(self.out4)
        print("out 5: {}".format(self.out5.shape))
        # self.out5 = torch.cat((self.out5, self.out1))

        self.out_final = self.finalconv(self.out5)
        print("out final: {}".format(self.out_final.shape))
        # self.out_final = torch.cat((self.out_final, x))

        return self.out_final
        ###################################################

In [7]:
# Set the maximum number of threads to prevent crash in Teaching Labs
torch.set_num_threads(5)

# Numpy random seed
npr.seed(seed)

# LOAD THE COLOURS CATEGORIES
colours = np.load(colours)[0]
num_colours = np.shape(colours)[0]

# LOAD THE MODEL
if model == "CNN":
    cnn = CNN(kernel, num_filters, num_colours)
elif model == "UNet":
    cnn = UNet(kernel, num_filters, num_colours)
else:  # model == "DUNet":
    cnn = DilatedUNet(kernel, num_filters, num_colours)

# LOSS FUNCTION
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(cnn.parameters(), lr = learn_rate)

# DATA
print("Loading data...")
(x_train, y_train), (x_test, y_test) = load_cifar10()

print("Transforming data...")
train_rgb, train_grey = process(x_train, y_train)
train_rgb_cat = get_rgb_cat(train_rgb, colours)
test_rgb, test_grey = process(x_test, y_test)
test_rgb_cat = get_rgb_cat(test_rgb, colours)

print("Done")

# Create the outputs folder if not created already
if not os.path.exists("outputs"):
    os.makedirs("outputs")

# Run validation only
if valid:
    if not checkpoint:
        raise ValueError("You need to give trained model to evaluate")

    print("Loading checkpoint...")
    cnn.load_state_dict(
        torch.load(checkpoint, map_location = lambda storage, loc: storage))
    img_path = "outputs/eval_%s.png" % model
    val_loss, val_acc = run_validation_step(cnn,
                                            criterion,
                                            test_grey,
                                            test_rgb_cat,
                                            batch_size,
                                            colours,
                                            img_path)
    print('Evaluating Model %s: %s' % (model, checkpoint))
    print('Val Loss: %.4f, Val Acc: %.1f%%' % (val_loss, val_acc))
    print('Sample output available at: %s' % img_path)
    exit(0)


Loading data...
data/cifar-10-batches-py.tar.gz
Transforming data...
Done


In [8]:
print("Beginning training ...")
if gpu:
    cnn.cuda()
start = time.time()

train_losses = []
valid_losses = []
valid_accs = []
for epoch in range(epochs):
    # Train the Model
    cnn.train()  # Change model to 'train' mode
    losses = []
    for i, (xs, ys) in enumerate(get_batch(train_grey,
                                           train_rgb_cat,
                                           batch_size)):
        images, labels = get_torch_vars(xs, ys, gpu)
        # Forward + Backward + Optimize
        optimizer.zero_grad()
        outputs = cnn(images)

        loss = compute_loss(criterion,
                            outputs,
                            labels,
                            batch_size = batch_size,
                            num_colours = num_colours)
        loss.backward()
        optimizer.step()
        losses.append(loss.data[0])

    # plot training images
    if plot:
        _, predicted = torch.max(outputs.data, 1, keepdim = True)
        plot(xs, ys, predicted.cpu().numpy(), colours,
             'outputs/train_%d.png' % epoch)

    # plot training images
    avg_loss = np.mean(losses)
    train_losses.append(avg_loss)
    time_elapsed = time.time() - start
    print('Epoch [%d/%d], Loss: %.4f, Time (s): %d' % (
        epoch + 1, epochs, avg_loss, time_elapsed))

    # Evaluate the model
    cnn.eval()  # Change model to 'eval' mode (BN uses moving mean/var).

    outfile = None
    if plot:
        outfile = 'outputs/test_%d.png' % epoch

    val_loss, val_acc = run_validation_step(cnn,
                                            criterion,
                                            test_grey,
                                            test_rgb_cat,
                                            batch_size,
                                            colours,
                                            outfile)

    time_elapsed = time.time() - start
    valid_losses.append(val_loss)
    valid_accs.append(val_acc)
    print('Epoch [%d/%d], Val Loss: %.4f, Val Acc: %.1f%%, Time(s): %d' % (
        epoch + 1, epochs, val_loss, val_acc, time_elapsed))

Beginning training ...
x: torch.Size([10, 1, 32, 32])
out 1: torch.Size([10, 32, 16, 16])
out 2: torch.Size([10, 64, 8, 8])
out 3: torch.Size([10, 64, 8, 8])
out 4: torch.Size([10, 32, 16, 16])
out 5: torch.Size([10, 24, 32, 32])
out final: torch.Size([10, 24, 32, 32])


/anaconda3/envs/Python2.7/lib/python2.7/site-packages/torch/nn/modules/upsampling.py:129: UserWarning: nn.Upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.{} is deprecated. Use nn.functional.interpolate instead.".format(self.name))


IndexError: invalid index of a 0-dim tensor. Use tensor.item() to convert a 0-dim tensor to a Python number

In [9]:

# # Plot training curve
# plt.plot(train_losses, "ro-", label = "Train")
# plt.plot(valid_losses, "go-", label = "Validation")
# plt.legend()
# plt.title("Loss")
# plt.xlabel("Epochs")
# plt.savefig("outputs/training_curve.png")

# if checkpoint:
#     print('Saving model...')
#     torch.save(cnn.state_dict(), checkpoint)

In [27]:
a = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
print(a.shape)
print(a)
print(a.transpose((0, 1)))

(3, 3)
[[1 2 3]
 [4 5 6]
 [7 8 9]]
[[1 2 3]
 [4 5 6]
 [7 8 9]]


In [8]:
import torch

torch.Tensor(1, 2)

tensor([[0.0000e+00, 4.6566e-10]])